# Census Data

In [104]:
import cenpy
import pandas as pd
import geopandas as gpd
acs = cenpy.remote.APIConnection("ACSDT5Y2015")

In [105]:
censusRace = acs.query(
    cols=["NAME","B02001_001E","B02001_002E"],
    geo_unit="block group:*",
    geo_filter={
                "state" : "42", 
                "county" : "101"
               },
).rename(columns={"B02001_001E": "totalPop","B02001_002E": "whitePop"}, errors="raise")

In [106]:
censusRace2 = censusRace
censusRace2['whitePop'] = pd.to_numeric(censusRace2['whitePop'], errors='ignore')
censusRace2['totalPop'] = pd.to_numeric(censusRace2['totalPop'], errors='ignore')
censusRace2['whitePercep'] = censusRace2['whitePop'] / censusRace2['totalPop'] 
censusRace2['nonwhitePercep'] = 1- censusRace2['whitePercep']
censusRace2.head()

,NAME,totalPop,whitePop,state,county,tract,block group,whitePercep,nonwhitePercep
0,"Block Group 1, Census Tract 81.02, Philadelphi...",858,0,42,101,008102,1,0.000000,1.000000
1,"Block Group 5, Census Tract 81.02, Philadelphi...",894,11,42,101,008102,5,0.012304,0.987696
2,"Block Group 3, Census Tract 81.02, Philadelphi...",543,0,42,101,008102,3,0.000000,1.000000
3,"Block Group 2, Census Tract 248, Philadelphia ...",1109,26,42,101,024800,2,0.023445,0.976555
4,"Block Group 1, Census Tract 2, Philadelphia Co...",2331,911,42,101,000200,1,0.390819,0.609181


In [107]:
censusRace2.to_csv('censusRaceData.csv', index=False) 

# 311 Complaints

In [108]:
complaint311 = pd.read_csv ("downloaded_public_cases_fc.csv")

In [109]:
ndService=['Abandoned Vehicle','Graffiti Removal','Vacant House or Commercial','Street Light Outage','Traffic Signal Emergency','Dead Animal in Street', 'Street Defect','Vacant Lot Clean-Up', 'Alley Light Outage', 'Street Trees', 'Abandoned Bike',
       'Complaint (Streets)', 'Street Paving', 'Tree Dangerous']

In [110]:
complaint311Filter = complaint311.query('service_name in @ndService')
complaint311Filter = complaint311Filter[["service_request_id","service_name","agency_responsible","lat","lon"]].dropna()

In [111]:
complaint311Filter['geometry'] = gpd.points_from_xy(complaint311Filter['lon'], complaint311Filter['lat'])
complaint311Filter = gpd.GeoDataFrame(complaint311Filter, geometry='geometry', crs="EPSG:4326")

In [112]:
complaint311Filter.count()

service_request_id    64523
service_name          64523
agency_responsible    64523
lat                   64523
lon                   64523
geometry              64523
dtype: int64

# Crime

In [113]:
import carto2gpd
import matplotlib
import numpy as np
from sklearn.neighbors import NearestNeighbors
import altair as alt
from vega_datasets import data
from altair import datum

In [114]:
url = "https://phl.carto.com/api/v2/sql"
table_name = "incidents_part1_part2"
where = "dispatch_date >= '2015-01-01' AND dispatch_date < '2016-01-01'"
crime = carto2gpd.get(url, table_name, where=where)

In [115]:
crime = crime.drop(columns=["objectid","dc_dist","psa","dc_key","ucr_general","point_x","point_y"])
crime["count"]=1

In [116]:
crime = crime.rename(columns={"text_general_code": "crimeType"}, errors="raise")

In [136]:
total = crime.groupby("crimeType").sum("count").sort_values(by="count",ascending=False)
total = total.reset_index()

alt.Chart(total).mark_bar().encode(
    alt.X('crimeType:N',axis=alt.Axis(labelAngle=90),sort='-y'),
    alt.Y('count:Q'),
    color = alt.value("#F79D46"))

In [119]:
selectedCrimeType = total[0:20].crimeType

In [159]:
selectedCrimeType

0                          All Other Offenses
1                              Other Assaults
2                                      Thefts
3                 Vandalism/Criminal Mischief
4                          Theft from Vehicle
5                                       Fraud
6              Narcotic / Drug Law Violations
7                        Burglary Residential
8               Aggravated Assault No Firearm
9                 DRIVING UNDER THE INFLUENCE
10                         Robbery No Firearm
11                         Disorderly Conduct
12                            Robbery Firearm
13                 Aggravated Assault Firearm
14                        Motor Vehicle Theft
15                   Burglary Non-Residential
16                                       Rape
17                          Weapon Violations
18    Other Sex Offenses (Not Commercialized)
19       Prostitution and Commercialized Vice
Name: crimeType, dtype: object

In [120]:
selectCrime = crime[crime['crimeType'].isin(selectedCrimeType)]
selectCrime.head()

,geometry,cartodb_id,dispatch_date_time,dispatch_date,dispatch_time,hour_,location_block,crimeType,count
0,POINT (-75.22059 39.91443),14,2015-01-15T08:58:00Z,2015-01-15,08:58:00,8,6600 BLOCK ESSINGTON AVE,Burglary Non-Residential,1
1,POINT (-75.22059 39.91443),15,2015-05-27T20:09:00Z,2015-05-27,20:09:00,20,6600 BLOCK ESSINGTON AVE,Aggravated Assault No Firearm,1
2,POINT (-75.22059 39.91443),23,2015-01-11T14:38:00Z,2015-01-11,14:38:00,14,6600 BLOCK ESSINGTON AVE,Robbery Firearm,1
3,POINT (-75.06663 40.04926),29,2015-04-15T11:14:00Z,2015-04-15,11:14:00,11,100 BLOCK S BROAD ST,Thefts,1
4,POINT (-75.16145 39.96233),53,2015-01-31T12:45:00Z,2015-01-31,12:45:00,12,1400 BLOCK SPRING GARDEN ST 210,Thefts,1


In [121]:
def get_xy_from_geometry(df): 
    x = df.geometry.centroid.x
    y = df.geometry.centroid.y
    return np.column_stack((x, y))

In [122]:
pointsWithSeg = gpd.read_file("pointsWithSeg.geojson")
pointsWithSeg.tail()

,fid,SEG_ID,pointId,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice,geometry
12312,24861,423467,012313,0.051546,0.011665,0.146067,0.052584,0.061044,0.014317,0.148602,0.332510,0.032989,0.000019,POINT (-75.19385 39.95761)
12313,24863,522448,012314,0.036714,0.003632,0.197326,0.015623,0.019405,0.003073,0.189704,0.381982,0.020216,0.000027,POINT (-75.15766 40.00221)
12314,24868,240821,012315,0.021332,0.000403,0.250894,0.013468,0.059427,0.008680,0.310452,0.258119,0.009212,0.001228,POINT (-75.16788 39.90467)
12315,24869,240822,012316,0.018760,0.001339,0.290828,0.009906,0.066447,0.007145,0.309186,0.222295,0.007647,0.000092,POINT (-75.16553 39.90436)
12316,24872,423473,012317,0.021787,0.000262,0.130251,0.057066,0.193078,0.027137,0.102466,0.345825,0.027737,0.000032,POINT (-75.19504 39.95630)


In [123]:
pointsWithSegXY = get_xy_from_geometry(pointsWithSeg.to_crs(epsg=3857))
crimeXY = get_xy_from_geometry(selectCrime.to_crs(epsg=3857))

nbrs = NearestNeighbors(n_neighbors=1)
nbrs.fit(pointsWithSegXY)

Dists, Indices = nbrs.kneighbors(crimeXY)

In [223]:
selectCrime["nearestSVPoint"] = Indices
selectCrime2 = selectCrime[['geometry','cartodb_id','crimeType','count','nearestSVPoint']].merge(pointsWithSeg.drop(columns=["geometry","fid","SEG_ID"]),how="left",left_on="nearestSVPoint",right_index=True)
selectCrime2 = selectCrime2.drop(columns="nearestSVPoint").rename(columns={"pointId": "nearestSVPoint"}, errors="raise")

D:\Miniconda_Python\envs\musa-550-fall-2021\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [226]:
#selectCrime2 = pd.DataFrame(selectCrime2.drop(columns="geometry"))
selectCrime2 = round(selectCrime2,3)
selectCrime2

,geometry,cartodb_id,crimeType,count,nearestSVPoint,wall,lives,building,infrastructure,road,sidewalk,sky,green,transportation,publicservice
0,POINT (-75.22059 39.91443),14,Burglary Non-Residential,1,002967,0.038,0.000,0.166,0.006,0.005,0.000,0.174,0.487,0.004,0.000
1,POINT (-75.22059 39.91443),15,Aggravated Assault No Firearm,1,002967,0.038,0.000,0.166,0.006,0.005,0.000,0.174,0.487,0.004,0.000
2,POINT (-75.22059 39.91443),23,Robbery Firearm,1,002967,0.038,0.000,0.166,0.006,0.005,0.000,0.174,0.487,0.004,0.000
3,POINT (-75.06663 40.04926),29,Thefts,1,008024,0.012,0.002,0.092,0.061,0.086,0.004,0.120,0.447,0.028,0.000
4,POINT (-75.16145 39.96233),53,Thefts,1,010759,0.030,0.006,0.219,0.004,0.047,0.006,0.235,0.302,0.069,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174718,POINT (-75.13372 39.99982),2599998,Motor Vehicle Theft,1,003466,0.022,0.001,0.271,0.011,0.029,0.006,0.231,0.318,0.024,0.000
174719,POINT (-75.01231 40.05670),2600020,Motor Vehicle Theft,1,004511,0.085,0.001,0.068,0.008,0.041,0.010,0.403,0.259,0.023,0.000
174720,POINT (-75.15006 39.91678),2600417,Motor Vehicle Theft,1,003720,0.085,0.001,0.097,0.032,0.088,0.026,0.338,0.221,0.024,0.000
174721,POINT (-75.07548 40.06314),2600421,Motor Vehicle Theft,1,010676,0.028,0.000,0.089,0.008,0.139,0.010,0.341,0.262,0.055,0.000


In [197]:
secondSelection = ["Prostitution and Commercialized Vice"]
selectCrime3 = selectCrime2[selectCrime2['crimeType'].isin(secondSelection)]

In [271]:
sampleCrime = selectCrime2.groupby('crimeType', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))
#sampleCrime = selectCrime3.groupby('crimeType', group_keys=False).meanO()
alt.data_transformers.enable('json')

selection = alt.selection_multi(fields=['crimeType'], bind='legend')

chart=alt.Chart(sampleCrime).transform_window(
    index='count()'
).transform_fold(
    ['wall', 'lives','building','infrastructure','road','sidewalk','sky','green','transportation']
).transform_joinaggregate(
     min='min(value)',
     max='max(value)',
     groupby=['key']
).transform_calculate(
    minmax_value=(datum.value-datum.min)/(datum.max-datum.min)
).mark_line(size=0.5).encode(
    x='key:N',
    y='minmax_value:Q',
    color=alt.condition(selection,
                        alt.Color('crimeType:N', scale=alt.Scale(scheme='category20b')),
                        alt.value('lightgray')),   
    opacity=alt.condition(selection, alt.value(1), alt.value(0.07)),   
    detail='index:N',
).properties(
    width=1000,
    height=500,
    title="Relationship between street component and crime type"
).add_selection(selection).interactive()

chart.encoding.x.title = 'Street Component'
chart.encoding.y.title = 'Normalized porportion'
chart

alt.Chart(...)